In [167]:
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
import math
import os
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn import metrics

In [170]:
def load_train_data(path):
    data = pd.read_csv(path)
    train_data = data.iloc[:,1:-1]
    print(Counter(train_data.dtypes))
    print("长度:",len(train_data.columns))
    print(len(train_data))
    return train_data, data.iloc[:,-1]
def load_testdata(path):
    data = pd.read_csv(path)
    data = data.iloc[:,1:]
    print(Counter(data.dtypes))
    print("长度:",len(train_data.columns))
    print(len(data))
    return data

In [171]:
x_train,y_train = load_train_data('./price/train.csv')

Counter({dtype('O'): 43, dtype('int64'): 33, dtype('float64'): 3})
长度: 79
1460


In [152]:
x_test = load_testdata('./price/test.csv')

Counter({dtype('O'): 43, dtype('int64'): 25, dtype('float64'): 11})
长度: 79
1459


In [153]:
data = pd.concat([x_train,x_test])

In [154]:
data = pd.get_dummies(data)

In [155]:
len(data)

2919

In [156]:
data.columns[data.isnull().sum() > 0]

Index(['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea'],
      dtype='object')

In [157]:
for column in list(data.columns[data.isnull().sum() > 0]):
    mean_val = data[column].mean()
    data[column].fillna(mean_val, inplace=True)

In [159]:
x_train = data.iloc[:1460]
x_test = data.iloc[1460:]

In [160]:
x_test

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,...,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,324.0,926.0,926,678,0,1604,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,1017.0,1280.0,1280,0,0,1280,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,546.0,546.0,546,546,0,1092,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1455,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,294.0,546.0,546,546,0,1092,...,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
1456,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,0.0,1224.0,1224,0,0,1224,...,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
1457,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,575.0,912.0,970,0,0,970,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [161]:
bst = XGBRegressor(seed = 1850)

In [162]:
bst.fit(x_train,y_train.values.ravel(),verbose=True)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=1850,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1850,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [165]:
preds = bst.predict(x_test)
preds

/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


array([129192.39 , 151508.56 , 183351.02 , ..., 171437.33 , 117077.766,
       218866.47 ], dtype=float32)

In [168]:
preds_train = bst.predict(x_train)

/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [169]:
metrics.r2_score(y_train.values,preds_train)

0.9994639347081627

In [172]:
type(preds)

numpy.ndarray

In [176]:
pre_df = pd.DataFrame({'id':list(range(1461,2920)),'SalePrice':preds})

In [178]:
pre_df.to_csv('./price/submission.csv',index=None)